In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import ipywidgets as widgets
from elm_data.step_5_label_elms.elm_labeling_app import BES_ELM_Labeling_App
from elm_data.data_tools import print_hdf5_contents

In [2]:
elm_label_file = './labeled_elms_v3.hdf5'
print_hdf5_contents(elm_label_file, print_datasets=True, max_groups=2)

Contents of labeled_elms_v3.hdf5
Group /: 0 datasets, 2 subgroups, and 1 attributes
  Attribute excluded_shots: shape (150,) dtype int64
Group /elms: 0 datasets, 2009 subgroups, and 0 attributes
Group /elms/000000: 0 datasets, 0 subgroups, and 3 attributes
  Attribute shot: value 192006 type <class 'numpy.int64'>
  Attribute t_start: value 3537.780029296875 type <class 'numpy.float64'>
  Attribute t_stop: value 3568.83994140625 type <class 'numpy.float64'>
Group /elms/000001: 0 datasets, 0 subgroups, and 3 attributes
  Attribute shot: value 192006 type <class 'numpy.int64'>
  Attribute t_start: value 3569.93994140625 type <class 'numpy.float64'>
  Attribute t_stop: value 3589.75986328125 type <class 'numpy.float64'>
Group /shots: 0 datasets, 89 subgroups, and 0 attributes
Group /shots/164791: 16 datasets, 0 subgroups, and 30 attributes
  Attribute bt_extremum: value -2.092060089111328 type <class 'numpy.float32'>
  Attribute bt_pos_phi: value False type <class 'numpy.bool_'>
  Attribut

In [3]:
plt.close('all')
widgets.Widget.close_all()

app = BES_ELM_Labeling_App(
    data_hdf5_file='/home/smithdr/ml/elm_data/step_4_shot_partial_data/data_v5.hdf5',
    elm_labels_hdf5_file=elm_label_file,
)

HDF5 data file: /home/smithdr/ml/elm_data/step_4_shot_partial_data/data_v5.hdf5
  Shots: 887
HDF5 labeled ELM file: /fusion/projects/diagnostics/bes/smithdr/edgeml/elm_data/step_5_label_elms/labeled_elms_v3.hdf5
  2009 ELMs from 89 shots (150 excluded shots)


In [4]:
widgets.AppLayout(
    left_sidebar=app.controls,
    center=app.fig.canvas,
    footer=app.status_label,
    pane_widths=['160px', 1, 0],
    pane_heights=[0, 1, '40px'],
)

AppLayout(children=(Label(value='0 ELMs for this shot | 1982 ELMs from 87 shots (144 skipped shots) | State: c…